# Introduction to GeoPandas


## Table of Contents

1. [GeoDataFrame](#GeoDataFrames)<br>
2. [Attributes and methods](#attributes)<br>
3. [Points vs Lines vs Polygons](#geometry)<br>
4. [Overlay](#overlay)<br>
5. [Buffer](#buffer)<br>
6. [Spatial relationships](#spatial)<br>
7. [London boroughs](#boroughs)<br>
    7.1. [Load geospatial data](#load1)<br>
    7.2. [Explore data](#explore1)<br>
    7.3. [Dissolve](#dissolve)<br>
    7.4. [Join](#join)<br>
8. [Open Street Map data (OSM)](#osm)<br>
    8.1. [Load data](#load2)<br>
    8.2. [Explore data](#explore2)<br>


<div class="alert alert-danger" style="font-size:100%">
If you are using <b>Watson Studio</b> to run the workshop you will need to add the project token to your notebook that you created earlier to be able to access the shape files from your Cloud Object Store (COS). 

Click the 3 dots at the top right side of the notebook to insert the project token. This will create a new cell in the notebook that you will need to run first before continuing with the rest of the notebook. If you are sharing this notebook you should remove this cell, else anyone can use you Cloud Object Storage from this project.

If you cannot find the new cell it is probably at the top of this notebook. Scroll up, run the cell and continue with the rest of the notebook below.

</div> 

In [ ]:
import pandas as pd
import geopandas as gpd

from shapely.geometry import Point, LineString, Polygon
import matplotlib.pyplot as plt

%matplotlib inline

<a id="GeoDataFrames"></a>
## 1. GeoDataFrame
GeoPandas extends two of its main data structures namely **GeoSeries** and **GeoDataFrame** from pandas. GeoSeries, much like pandas' *Series* is a vector in which each entry represents one or more shapes corresponding to a row. GeoDataFrame, much like pandas' *DataFrame* is a two dimensional data structure that has a column which is the GeoSeries along with other information. The GeoSeries column within the GeoDataFrame is referred to as the *geometry* of it. 



Below the latitude and longitude of 5 cities are used to create a `POINT` geometry variable that is used to create a `GeoDataFrame` from a `DataFrame`: 

In [ ]:
df = pd.DataFrame({'city':       ['London','Manchester','Birmingham','Leeds','Glasgow'],
        'population': [9787426,  2553379,     2440986,    1777934, 1209143],
        'area':       [1737.9,   630.3,       598.9,      487.8,   368.5 ],
        'latitude':   [51.50853, 53.48095,    52.48142,   53.79648,55.86515],
        'longitude':  [-0.12574, -2.23743,    -1.89983,   -1.54785,-4.25763]})

df.head()

In [ ]:
df['point']  = list(zip(df.longitude, df.latitude))
df['geometry'] = df['point'].apply(Point)
df.head()

In [ ]:
df = df.drop('point', 1)
cities = gpd.GeoDataFrame(df, geometry='geometry')
cities.head()

<a id="attributes"></a>
## 2. Attributes and methods 
But there are additional methods you can use from the [geopandas documentation](http://geopandas.org/data_structures.html#overview-of-attributes-and-methods)):

We can explore a few of these with the cities data:


As `cities` is still a DataFrame you can apply the same data manipulations, for instance:

In [ ]:
cities['population'].mean()

In [ ]:
cities['area'].min()

In [ ]:
cities['density'] = cities['population']/cities['area']
cities

In [ ]:
cities.area

In [ ]:
cities.total_bounds

In [ ]:
cities.geom_type

In [ ]:
cities.distance(cities.geometry[0])

In [ ]:
cities2 = cities.copy()
cities2 = cities2.drop([2,3])
cities2.head()

In [ ]:
cities2.area

In [ ]:
cities2.bounds

In [ ]:
cities2.geom_type

In [ ]:
cities2.centroid

In [ ]:
cities2.representative_point()

<a id="geometry"></a>
## 3. Points vs Lines vs Polygons
For the other attributes and methods we need some more data. 






* A point by squeezing out the geometry

In [ ]:
lon_point = cities.loc[cities['city'] == 'London', 'geometry'].squeeze()
man_point = cities.loc[cities['city'] == 'Manchester', 'geometry'].squeeze()
birm_point = cities.loc[cities['city'] == 'Birmingham', 'geometry'].squeeze()
leeds_point = cities.loc[cities['city'] == 'Leeds', 'geometry'].squeeze()

* A line between 2 cities by creating a LineString between 2 points

In [ ]:
lon_man_line = gpd.GeoSeries(LineString([lon_point, man_point]))
man_birm_line = gpd.GeoSeries(LineString([man_point, birm_point]))
birm_lon_line = gpd.GeoSeries(LineString([birm_point,lon_point]))
leeds_man_line = gpd.GeoSeries(LineString([leeds_point, man_point]))
birm_leeds_line = gpd.GeoSeries(LineString([birm_point,leeds_point]))

* A polygon between 3 cities by creating a Polygon between 3 points

In [ ]:
Polygon([[lon_point.x,lon_point.y],[man_point.x,man_point.y],[lon_point.x,lon_point.y]])
lon_man_birm_polygon = gpd.GeoSeries(Polygon([[lon_point.x,lon_point.y],[man_point.x,man_point.y],[birm_point.x,birm_point.y],[lon_point.x,lon_point.y]]))
leeds_man_birm_polygon = gpd.GeoSeries(Polygon([[leeds_point.x,leeds_point.y],[man_point.x,man_point.y],[birm_point.x,birm_point.y]]))

And plot all of them together:

In [ ]:
fig, (poly1,poly2) = plt.subplots(ncols=2, sharex=True, sharey=True)

lon_man_birm_polygon.plot(ax=poly1, color='lightblue', edgecolor='black',alpha=0.5);
lon_man_line.plot(ax=poly1,color='violet',alpha=0.5);
man_birm_line.plot(ax=poly1,color='blue',alpha=0.5);
birm_lon_line.plot(ax=poly1,color='green',alpha=0.5);

leeds_man_birm_polygon.plot(ax=poly2, color='yellow', edgecolor='black',alpha=0.5);
leeds_man_line.plot(ax=poly2,color='red',alpha=0.5);
man_birm_line.plot(ax=poly2,color='blue',alpha=0.5);
birm_leeds_line.plot(ax=poly2,color='green',alpha=0.5);

With these new shapes let's explore some more methods.

<a id="overlay"></a>
## 4. Overlay
To observe interactions between more than one datasets, we use the overlay library. *union*, *difference*, *symmetrical difference* and *intersection* are some of the operations that can be performed. 


Below is a graph showing the union of the two polygons displayed in teh graph above.

In [ ]:
poly1 = gpd.GeoDataFrame({'geometry': lon_man_birm_polygon})
poly2 = gpd.GeoDataFrame({'geometry': leeds_man_birm_polygon})

gpd.overlay( poly1, poly2, how='union').plot(color='red',alpha=0.5);

<a id="buffer"></a>
## 5. Buffer


* Circles around the cities by adding a buffer around the points
Polygons can be of any shape as you will see later in the workshop, using circles here as a quick example. 

In [ ]:
cities1 = cities[0:1].copy()
cities1.head()

In [ ]:
base = cities1.buffer(3).plot(color='blue',alpha=0.5);
cities1.buffer(2).plot(ax=base,color='green',alpha=0.5);
cities1.buffer(1).plot(ax=base,color='yellow',alpha=0.5);
cities1.plot(ax=base,color='red',alpha=0.5);

<a id="spatial"></a>
## 6. Spatial relationships

What can you do with geospatial relationships?  

There are several functions to check geospatial relationships between geometries: `equals`, `contains`, `crosses`, `disjoint`,`intersects`,`overlaps`,`touches`,`within` and `covers`. These all use the `shapely` package about which you can read more [here](https://shapely.readthedocs.io/en/stable/manual.html#predicates-and-relationships) and some more background on spatial relationships [here](https://en.wikipedia.org/wiki/Spatial_relation).

A few examples:

In [ ]:
cities.head()

In [ ]:
cities2.head()

In [ ]:
cities.contains(cities2)

In [ ]:
cities2.contains(lon_point)

In [ ]:
cities2[cities2.contains(lon_point)]

In [ ]:
cities2[cities2.contains(man_point)]

The inverse of `contains`:

In [ ]:
cities[cities.within(cities2)]

In [ ]:
cities2.intersects(lon_man_line)

In [ ]:
cities2[cities2.crosses(man_birm_line)]

In [ ]:
cities2[cities2.disjoint(lon_point)]

<a id="boroughs"></a>
## 7. London boroughs



<a id="load1"></a>
### 7.1 Load geospatial data

Geospatial data comes in many formats, but with GeoPandas you can read most files with just one command. For example this geojson file with the London boroughs: 



In [ ]:
# load data from a url
boroughs = gpd.read_file("https://skgrange.github.io/www/data/london_boroughs.json")
boroughs.head()

<a id="explore1"></a>
### 7.2 Explore  data

In [ ]:
boroughs.plot();

Adding a column will colour the map based on the classes in this column:

In [ ]:
boroughs.plot(column='code');

In [ ]:
boroughs.plot(column='area_hectares');

<a id="dissolve"></a>
### 7.3 Dissolve

The boroughs are made up of many districts that you might want to combine. For this example this can be done by adding a new column and then use `.dissolve()`:

In [ ]:
boroughs['all'] = 1
allboroughs = boroughs.dissolve(by='all',aggfunc='sum')
allboroughs.head()

In [ ]:
allboroughs.plot();

To change the size of the map and remove the box around the map, run the below:

In [ ]:
[fig, ax] = plt.subplots(1, figsize=(10, 6))
allboroughs.plot(ax=ax);
ax.axis('off');

<a id="join"></a>
### 7.4 Join

Let's combine the data from the Pandas notebook with the boroughs GeoDataFrame:

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/IBMDeveloperUK/crime-data-workshop/master/data/london-borough-profiles.csv',encoding = 'unicode_escape')

In [ ]:
df.head()

In [ ]:
boroughs.head()

The columns to join the two tables on are `code` and `Code`. To use the [`join` method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html), first the index of both tables has to be set to this column. 

The below adds the columns from `df` to `boroughs`:

In [ ]:
boroughs = boroughs.set_index('code').join(df.set_index('Code'))
boroughs.head()

Below is a map that shows two regions: Inner and Outer London


In [ ]:
boroughs2 = boroughs.dissolve(by='Inner/_Outer_London',aggfunc='mean')
boroughs2.head()

In [ ]:
[fig, ax] = plt.subplots(1, figsize=(10, 6))
boroughs2.plot(column='id', cmap='Paired', linewidth=0.5, edgecolor='black', legend=False, ax=ax);
ax.axis('off');

Below is a map of the average gender pay gap for each borough.


In [ ]:
boroughs['paygap'] = \
    ((boroughs['Gross_Annual_Pay_-_Male_(2016)'] - boroughs['Gross_Annual_Pay_-_Female_(2016)'])/ \
    boroughs['Gross_Annual_Pay_-_Male_(2016)']) * 100

[fig,ax] = plt.subplots(1, figsize=(12, 8))

boroughs.plot(ax=ax, color="lightgrey", edgecolor='black', linewidth=0.5)

boroughs.dropna().plot(column='paygap', cmap='Reds', edgecolor='black', linewidth=0.5,
               legend=True, ax=ax, scheme='equal_interval');
ax.axis('off');
ax.set_title('Gender pay gap in London (2016)');


<a id="osm"></a>
## 8. Open Street Map data (OSM)

<a id="load2"></a>
### 8.1 Load OSM data

Data is downloaded from http://download.geofabrik.de/europe/great-britain.html and a more detailed decription of the data is [here](http://download.geofabrik.de/osm-data-in-gis-formats-free.pdf).

The following code was run to extract vector data around just london and import it to a local file. 

``xmin, ymin, xmax, ymax = london.total_bounds <br>
pois_all = gpd.read_file("../data/england-latest-free/gis_osm_pois_free_1.shp") <br>
pois = pois_all.cx[xmin:xmax, ymin:ymax] <br>
pois.to_file("../data/london_pois.shp") <br>``

The data format is a shape file that consists of several files combined into one zip file that can be read directly with GeoPandas:

In [ ]:
# define the helper function 
def download_file_to_local(project_filename, local_file_destination=None, project=None):
    """
    Uses project-lib to get a bytearray and then downloads this file to local.
    Requires a valid `project` object.
    
    Args:
        project_filename str: the filename to be passed to get_file
        local_file_destination: the filename for the local file if different
        
    Returns:
        0 if everything worked
    """
    
    project = project
    
    # get the file
    print("Attempting to get file {}".format(project_filename))
    _bytes = project.get_file(project_filename).read()
    
    # check for new file name, download the file
    print("Downloading...")
    if local_file_destination==None: local_file_destination = project_filename
    
    with open(local_file_destination, 'wb') as f: 
        f.write(bytearray(_bytes))
        print("Completed writing to {}".format(local_file_destination))
        
    return 0

In [ ]:
download_file_to_local('london_pois.zip', project=project)
zipfile = "zip://./london_pois.zip!london_pois/london_pois.dbf"
pois = gpd.read_file(zipfile)


In [ ]:
pois.head()

<a id="explore2"></a>
### 8.2 Explore OSM data

In [ ]:
pois.size

In [ ]:
pois['fclass'].unique()

In [ ]:
pois.plot(column='fclass');

Let's count and plot the number of pubs by borough by:

* checking the coordinate systems of the maps to combine. They need to be the same to use them together.
* extracting the pubs from the `pois` DataFrame
* joining the tables into a temporary table
* counting the number of pubs in each borough
* merging this new table back into the `boroughs` DataFrame

In [ ]:
pois[pois.fclass=='pub'].plot(column='fclass');

The coordinate reference system (CRS) determines how the two-dimensional (planar) coordinates of the geometry objects should be related to actual places on the (non-planar) earth.

In [ ]:
print(pois.crs)
print(boroughs.crs)

In [ ]:
pubs = pois[pois['fclass']=='pub']
pubs.head()

In [ ]:
pubs2 = gpd.sjoin(boroughs,pubs) 
pubs2.head()

In [ ]:
pubs3 = pd.pivot_table(pubs2,index='name_left',columns='fclass',aggfunc={'fclass':'count'})
pubs3.columns = pubs3.columns.droplevel()
pubs3 = pubs3.reset_index()
pubs3.head()

In [ ]:
boroughs = boroughs.merge(pubs3, left_on='name',right_on='name_left')
boroughs = boroughs.drop(columns='name_left')
boroughs.head()

In [ ]:
[fig,ax] = plt.subplots(1, figsize=(12, 8))

boroughs.plot(column='pub',cmap='Blues', edgecolor='black', linewidth=0.5, 
              legend=True, ax=ax, scheme='equal_interval');
ax.axis('off');
ax.set_title('Pubs in London');



A different way to visualize this is with a heatmap:

In [ ]:
import geoplot 
[fig,ax] = plt.subplots(1, figsize=(12, 8))

geoplot.kdeplot(
    pubs, clip=boroughs.geometry, n_levels=10, 
    shade=True, cmap='Greens', ax=ax)
geoplot.polyplot(boroughs, ax=ax, alpha=1, edgecolor='black', linewidth=0.5)

Below is a map that only shows all points of one of the POI classes for one of the boroughs


In [ ]:
hotels = pois[pois['fclass']=='hotel']
citylondon = boroughs.loc[boroughs['name'] == 'City of London', 'geometry'].squeeze()
cityhotels = hotels[hotels.within(citylondon)]

[fig,ax] = plt.subplots(1, figsize=(12, 8))
base = boroughs.plot(color='lightblue', edgecolor='black',ax=ax);
cityhotels.plot(ax=ax, marker='o', color='red', markersize=8);
ax.axis('off');


Below is a map showing a new POI class being added to the boroughs table.

In [ ]:
hotels2 = gpd.sjoin(boroughs,hotels) 
hotels3 = pd.pivot_table(hotels2,index='name_left',columns='fclass',aggfunc={'fclass':'count'})
hotels3.columns = hotels3.columns.droplevel()
hotels3 = hotels3.reset_index()

boroughs = boroughs.merge(hotels3, left_on='name',right_on='name_left')
boroughs = boroughs.drop(columns='name_left')

[fig,ax] = plt.subplots(1, figsize=(12, 8))

boroughs.plot(column='hotel',cmap='Blues', edgecolor='black', linewidth=0.5, 
              legend=True, ax=ax, scheme='quantiles');
ax.axis('off');
ax.set_title('Hotels in London');


### Author
Margriet Groenendijk is a Data & AI Developer Advocate for IBM. She develops and presents talks and workshops about data science and AI. She is active in the local developer communities through attending, presenting and organising meetups. She has a background in climate science where she explored large observational datasets of carbon uptake by forests during her PhD, and global scale weather and climate models as a postdoctoral fellow. 

Samaya Madhavan is an Advisory Software Engineer with IBM where she currently publishes content that are related to machine learning and deep learning. She is also a full stack software developer, experienced in offering AI based solutions within the healthcare domain. Samaya has her Bachelor of Engineering in Computer Science from College of Engineering, Guindy and her Master of Science in Computer Science from University of Texas at Arlington. She is an ardent learner and a very passionate algorithm solver.

Copyright © 2019 IBM. This notebook and its source code are released under the terms of the MIT License.